## Chargement de la base de données.

In [ ]:
import pandas as pd

df= pd.read_excel("/Users/mathias/Desktop/Stat'App/ENSAE_Vietnam_geo-analysis_database/1. Facts/Districts data.xlsx")

df.head()

## Scaling

In [ ]:
#Détermination des colonnes non numériques

num_col = []
non_num_col = []
columns=df.columns.values
for i in range(len(df.dtypes)):
    typ=df.dtypes[i]
    if typ == 'float64':
        num_col.append(columns[i])
    elif typ == 'int64':
        num_col.append(columns[i])
    else :
        non_num_col.append(columns[i])
        
print(num_col)

In [ ]:
print(non_num_col)

In [ ]:
# On supprime 

In [ ]:
dfb=df

In [ ]:
dfb.head()

In [ ]:
num_col[2:]

In [ ]:
dfb[num_col[2:]]

On standardise la base de données.

### Avant la standardisation

In [ ]:
import seaborn as sns
#On regarde que pour les 5 premières colonnes
sns.pairplot(dfb[num_col[:5]])


### Après la standardisation

In [ ]:
import sklearn.preprocessing as proc
dfstd = pd.DataFrame(proc.StandardScaler().fit_transform(dfb[num_col[2:]]))
#on ne transforme pas les id
dfstd.columns = num_col[2:]
# Question devons-nous aussi ne pas standardiser d'autres id.
dfstd['OBJECTID']=df['OBJECTID']
dfstd['ID']=df['ID']

#Suppression de la géographie.
#dfstd=dfstd.drop(['AREA', 'grid_code','Shape__Area','Shape__Length'], axis=1)

In [ ]:
dfstd.head()

In [ ]:
sns.pairplot(dfstd[num_col[:5]])

# Détermination du nombre optimal de cluster

## Kmeans

#### Elbow method

In [ ]:
#Elbow Method
import matplotlib.pyplot as plt
from scipy.cluster.vq import kmeans, vq
distortions = []
num_clusters = range(1, 15)

# Create a list of distortions from the kmeans function
for i in num_clusters:
    cluster_centers, distortion = kmeans(dfstd,i)
    distortions.append(distortion)

# Create a data frame with two lists - num_clusters, distortions
elbow_plot = pd.DataFrame({'num_clusters': num_clusters, 'distortions': distortions})

# Creat a line plot of num_clusters and distortions
sns.lineplot(x='num_clusters', y='distortions', data = elbow_plot)
plt.xticks(num_clusters)
plt.show()

Il semble que le nombre optimal de cluster avec une méthode de k-means soit de l'ordre de 3-5;

## Clustering hierrchique ascendant 

Methode ward

In [ ]:
 # Import the fcluster and linkage functions
from scipy.cluster.hierarchy import fcluster, linkage

# Use the linkage() function
distance_matrix = linkage(dfstd, method = 'ward', metric = 'euclidean')


In [ ]:
# Dendogramme
# Pour faire les dendogrammes

# Import the dendrogram function
from scipy.cluster.hierarchy import dendrogram

# Create a dendrogram
dn = dendrogram(distance_matrix)

# Display the dendogram
plt.title("Dendogramme avec method = 'ward'")
plt.show()

Methode complete

In [ ]:
# Use the linkage() function
distance_matrix_complete = linkage(dfstd, method = 'complete', metric = 'euclidean')

dn = dendrogram(distance_matrix)

# Display the dendogram
plt.title("Dendogramme avec method = 'complete'")
plt.show()

Methode average

In [ ]:
# Use the linkage() function
distance_matrix_complete = linkage(dfstd, method = 'average', metric = 'euclidean')

dn = dendrogram(distance_matrix)

# Display the dendogram
plt.title("Dendogramme avec method = 'average'")
plt.show()

Methode single

In [ ]:
# Use the linkage() function
distance_matrix_complete = linkage(dfstd, method = 'single', metric = 'euclidean')

dn = dendrogram(distance_matrix)

# Display the dendogram
plt.title("Dendogramme avec method = 'single'")
plt.show()

# -----------------------------------------------------------------------------------------

# Mise à plat sur une carte

In [ ]:
dfstd['OBJECTID']=df['OBJECTID']
dfstd['ID']=df['ID']

In [ ]:
dfcluster= dfstd

In [ ]:
from scipy.cluster.vq import kmeans, vq

cluster_centers, distortion = kmeans(dfstd,7)

dfcluster['cluster_labels'], distortion_list = vq(dfstd,cluster_centers)

In [ ]:
dfcluster['District Name']= df['NAME']


In [ ]:
dfcluster.head()

In [ ]:
dfcluster=dfcluster[['ID','cluster_labels','District Name']]

In [ ]:
dfcluster.head()

### Geopandas

In [ ]:
geo = gpd.read_file('/Users/mathias/Downloads/vietnam_district_border_polygon.csv')
# The Data was found here : http://opendata.hcmgis.vn/layers/geonode_data:geonode:vietnam_district_border_polygon
# We only need to care about the Name, Province and geometry column
geo = geo[['Ten','Province', 'the_geom']]
geo.sample(10)

In [ ]:
geo.columns= ['Ten', 'Province','geometry']

In [ ]:
geo.head()

In [ ]:
type(geo['geometry'])

In [ ]:
from shapely import wkt

geo['geometry'] = geo['geometry'].apply(wkt.loads)
my_geo_df = gpd.GeoDataFrame(geo, geometry='geometry')

In [ ]:
my_geo_df.head()

In [ ]:
my_geo_df['geometry']

In [ ]:
my_geo_df.plot()

In [ ]:
geodata = gpd.GeoDataFrame(my_geo_df.merge(dfcluster, how='inner', left_on=['Ten'], right_on=['District Name']))
geodata.sample(10)

In [ ]:
type(geodata)

In [ ]:
fig = geodata.plot(figsize=(100,100),column=geodata['cluster_labels'])

In [ ]:
fig.figure.savefig('Clustering_Kmeans_7_avecGeo.png')

# Clustering hierarchique

In [ ]:
dfcluster2= dfstd

In [ ]:
dfcluster2=dfcluster.drop(['District Name'], axis=1)

In [ ]:

# Hierarchical clustering big picture
from scipy.cluster.hierarchy import fcluster, linkage


# Fit the data into a hierarchical clustering algorithm
distance_matrix = linkage(dfcluster2, 'ward')

# Assign cluster labels to each row of data
dfcluster2['cluster_labels'] = fcluster(distance_matrix, 7, criterion='maxclust')



In [ ]:
dfcluster2['District Name']= df['NAME']

In [ ]:
dfcluster2=dfcluster2[['ID','cluster_labels','District Name']]
dfcluster2.head()

In [ ]:
geodata = gpd.GeoDataFrame(my_geo_df.merge(dfcluster2, how='inner', left_on=['Ten'], right_on=['District Name']))
geodata.sample(10)

In [ ]:

fig = geodata.plot(figsize=(50,50),column=geodata['cluster_labels'],legend=True)

In [ ]:
fig.figure.savefig('clustering_ACH_7_avecGeo.png')

In [ ]:
# Stat Desc pour les différents clusters.

In [ ]:
dfcc=df

In [ ]:
dfcc['cluster_label'] = dfcluster2['cluster_labels']

In [ ]:
dfcc

In [ ]:
data = dfcc.groupby('cluster_label').describe()

In [ ]:
data.to_csv()